In [2]:
import pandas as pd
df = pd.read_csv("../data_csv/NASA.csv")
df.info()
df.columns

dt = pd.to_datetime(df.iloc[:, 0], errors="coerce")

mask = dt.dt.year.isin([2013, 2014])

mask.sum(), dt[mask].min(), dt[mask].max()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 7565 non-null   object
 1   start_time           7565 non-null   object
 2   ambient_temperature  7565 non-null   int64 
 3   battery_id           7565 non-null   object
 4   test_id              7565 non-null   int64 
 5   uid                  7565 non-null   int64 
 6   filename             7565 non-null   object
 7   Capacity             2794 non-null   object
 8   Re                   1956 non-null   object
 9   Rct                  1956 non-null   object
dtypes: int64(3), object(7)
memory usage: 591.1+ KB


/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_55087/1881772271.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(df.iloc[:, 0], errors="coerce")


(np.int64(0), NaT, NaT)

start time을 문자여로 바꾸고자 함 

In [3]:
import ast
import pandas as pd

def parse_start_time(x):
    try:
        t = ast.literal_eval(x)
        return pd.Timestamp(
            int(t[0]), int(t[1]), int(t[2]),
            int(t[3]), int(t[4]), int(float(t[5]))
        )
    except:
        return pd.NaT

df["timestamp"] = df["start_time"].apply(parse_start_time)

In [4]:
df["timestamp"].notna().sum()
df["timestamp"].min(), df["timestamp"].max()

(NaT, NaT)

In [5]:
df["start_time"].iloc[0]

'[2010.       7.      21.      15.       0.      35.093]'

In [6]:
import re
import pandas as pd

def parse_start_time(s: str):
    if pd.isna(s):
        return pd.NaT

    # 문자열에서 숫자(정수/소수/지수표기)만 추출
    nums = re.findall(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?', str(s))
    if len(nums) < 6:
        return pd.NaT

    y, mo, d, h, mi, sec = map(float, nums[:6])

    # 초(sec)에 소수점이 있을 수 있음
    sec_int = int(sec)
    micro = int(round((sec - sec_int) * 1_000_000))

    return pd.Timestamp(
        year=int(y), month=int(mo), day=int(d),
        hour=int(h), minute=int(mi), second=sec_int, microsecond=micro
    )

df["timestamp"] = df["start_time"].apply(parse_start_time)

In [7]:
df["timestamp"].notna().sum(), df["timestamp"].min(), df["timestamp"].max()

(np.int64(7565),
 Timestamp('2008-04-02 13:08:17.921000'),
 Timestamp('2010-09-30 15:32:33.078000'))

In [8]:
(df["timestamp"].dt.year.isin([2013, 2014])).sum()

np.int64(0)